In [6]:
import os
os.chdir('C:/Users/sinha/OneDrive/Desktop/MS - Big Data/Python/Assignment 1/movies1')

In [17]:
import pandas as pd
import numpy as np
import sklearn.metrics as sk
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.metrics import pairwise_distances
from scipy.spatial.distance import cosine, correlation
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import jaccard_similarity_score
from numpy import ndarray
from scipy import spatial
from sklearn.metrics.pairwise import linear_kernel
from scipy.spatial import distance
import matplotlib.pyplot as plt

In [8]:
###load dataset - user rating
user_rating_cols = ['user_id', 'movie_id', 'rating', 'timestamp']
user_ratings = pd.read_csv('u.data', sep = '\t', names = user_rating_cols)


In [9]:
###load dataset - movie genre

movie_cols = ['movie_id', 'title', 'release_date', 'Blank', 'imdb_url','unknown', 'Action', 'Adventure',
'Animation', 'Children', 'Comedy', 'Crime', 'Documentary', 'Drama', 'Fantasy','Film-Noir', 'Horror', 'Musical', 'Mystery', 'Romance', 'Sci-Fi', 'Thriller', 'War', 'Western']
movies = pd.read_csv('u.item', sep = '|', names = movie_cols, encoding='latin-1')
df = movies


In [10]:
####1. Similarity between 2 users

import warnings
warnings.filterwarnings("ignore")

#merging user_rating and movie data sets
df2 = pd.merge(user_ratings,movies)

def user_similarity_metric(userid1, userid2, df2):
    Userid1_rating = df2[df2['user_id'] == userid1]
    Userid2_rating = df2[df2['user_id'] == userid2]
    intersected_df = pd.merge(Userid1_rating, Userid2_rating, on=['movie_id'])
    count = len (intersected_df.index)

    class user_sim:
        def __init__(self, userid1,userid2):
            self.__userid1 = userid1
            self.__userid2 = userid2

        def getuserid1(self):
            return self.__userid1 

        def getuserid2(self):
            return self.__userid2

        def get_Jaccard_Sim(self):
            return (jaccard_similarity_score(intersected_df['rating_x'], intersected_df['rating_y']))

        def get_Euclidean_distance(self):
            return(distance.euclidean(intersected_df['rating_x'], intersected_df['rating_y']))

        def get_manhattan_distance(self):
            return(distance.cityblock(intersected_df['rating_x'], intersected_df['rating_y']))

        def get_cosine_similarity(self):
            return(1-(distance.cosine(intersected_df['rating_x'], intersected_df['rating_y'])))

        def get_pearson_similarity(self):
            return(np.corrcoef(intersected_df['rating_x'], intersected_df['rating_y'])[0,1])


    def main():
        user_sim_score = user_sim(userid1,userid2)
        print('Computing user similarity between userid', user_sim_score.getuserid1(), 'and userid', user_sim_score.getuserid2())
        print('\nBoth users have rated', count, 'movies in common!!!')
        print('\nThe similarity scores are as follows:')

        
        print('1. Manhattan Distance:', round(user_sim_score.get_manhattan_distance(),3))
        print('2. Euclidean Distance:', round(user_sim_score.get_Euclidean_distance(),3))
        print('3. Jacard Similarity:', round(user_sim_score.get_Jaccard_Sim(),3))
        print('4. Cosine Similarity:', round(user_sim_score.get_cosine_similarity(),3))
        print('5. Pearson Similarity:', round(user_sim_score.get_pearson_similarity(),3))      
        
        Man = user_sim_score.get_manhattan_distance()
        Euc = user_sim_score.get_Euclidean_distance()
        Jac = user_sim_score.get_Jaccard_Sim()
        Cos = user_sim_score.get_cosine_similarity()
        Pea = user_sim_score.get_pearson_similarity()
        
        graph = {'Similarity_metric': ['Manhattan','Euclidean', 'Jaccard Sim', 'Cosine Sim', 'Pearson Sim'],
            'Score': [Man, Euc, Jac, Cos, Pea]}
    
 
        graph_df = pd.DataFrame(graph, columns = ['Similarity_metric', 'Score'])
        
        ax = graph_df.plot.bar(rot=0)
        
        plt.show()
    
    main()
    return main



In [11]:
#### 2. computing movie similarity between 2 movies

#a. Replacing 0 in all genre columns with blank

genre = ['Action','Adventure','Animation', 'Children', 'Comedy', 'Crime', 'Documentary', 'Drama', 'Fantasy','Film-Noir', 'Horror', 'Musical', 'Mystery', 'Romance', 'Sci-Fi', 'Thriller', 'War', 'Western']

for genres in genre:
    df[genre] = df[genre].replace(0,'')
    
#b. Replacing 1 in all genre columns with genre

df['Action'].replace(1,'Action', inplace = True)
df['Adventure'].replace(1,'Adventure', inplace = True)
df['Animation'].replace(1,'Animation', inplace = True)
df['Children'].replace(1,'Children', inplace = True)
df['Comedy'].replace(1,'Comedy', inplace = True)
df['Crime'].replace(1,'Crime', inplace = True)
df['Documentary'].replace(1,'Documentary', inplace = True)
df['Drama'].replace(1,'Drama', inplace = True)
df['Fantasy'].replace(1,'Fantasy', inplace = True)
df['Film-Noir'].replace(1,'Film-Noir', inplace = True)
df['Horror'].replace(1,'Horror', inplace = True)
df['Musical'].replace(1,'Musical', inplace = True)
df['Mystery'].replace(1,'Mystery', inplace = True)
df['Romance'].replace(1,'Romance', inplace = True)
df['Sci-Fi'].replace(1,'Sci-Fi', inplace = True)
df['Thriller'].replace(1,'Thriller', inplace = True)
df['War'].replace(1,'War', inplace = True)
df['Western'].replace(1,'Western', inplace = True)

#c. Adding a column with combined genres

def combined_genres(row):
    return row['Action'] + " " + row['Adventure'] + " " + row['Animation'] + " " + row['Children'] + " " + row['Comedy'] + " " + row['Crime'] + " " + row['Documentary'] + " " + row['Drama'] + " " + row['Fantasy'] + " " + row['Film-Noir'] + " " + row['Horror'] + " " + row['Musical'] + " " + row['Mystery'] + " " + row['Romance'] + " " + row['Sci-Fi'] + " " + row['Thriller'] + " " + row['War'] + " " + row['Western']

df["combined_genres"] = df.apply(combined_genres, axis = 1)
   


In [12]:
#d. slicing required columns

sim_btwn_movies = df[['movie_id', 'title',  'combined_genres']]

#e. Extracting user defined movie_ids rows and concatenating in a new df


def similarity_btwn_2movies (movie_id1, movie_id2, sim_btwn_movies):
    from numpy import ndarray
    movie1_genre = sim_btwn_movies[sim_btwn_movies['movie_id'] == movie_id1]
    movie2_genre = sim_btwn_movies[sim_btwn_movies['movie_id'] == movie_id2]

    df_row = pd.concat([movie1_genre, movie2_genre])

    #f. Computing cosine similarity
    cv1 = CountVectorizer()
    count_matrix1 = cv1.fit_transform(df_row["combined_genres"])
    cosine_sim1 = cosine_similarity(count_matrix1)

    cosine_sim1 = (cosine_sim1)[0,1]

    print('Cosine Similarity between movie_id', movie_id1, 'and movie_id', movie_id2, 'is', round(cosine_sim1,3) )
    print('\nMovie Genre details follows: \n')
    print (df_row)
    return cosine_sim1



In [13]:
####3. Getting N most similiar movies for target movie

#Ignore english words like a, an etc from combined genre
tfidf = TfidfVectorizer(stop_words='english')

# generating fit matrix for computing similarity
tfidf_matrix = tfidf.fit_transform(df['combined_genres'])

# Compute the cosine similarity matrix
cosine_similarity = linear_kernel(tfidf_matrix, tfidf_matrix)

#Getting index of target movie
indices = pd.Series(df.index, index=df['title']).drop_duplicates()

def title_from_movie_id(movie_id):
    return df[df.movie_id == movie_id]["title"].values[0]

def N_similiar_movies(title, cosine_similarity=cosine_similarity):
    # Get the index of the movie that matches the title
    idx = indices[title]

    # Get the pairwsie similarity scores of all movies with that movie
    sim_scores = list(enumerate(cosine_similarity[idx]))

    # Sort the movies based on the similarity scores
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)

    # Get the scores of the 10 most similar movies

    sim_scores = sim_scores[0:n_movies]

    # Get the movie indices
    movie_indices = [i[0] for i in sim_scores]

    # Return the top 10 most similar movies
    return df['title'].iloc[movie_indices]

In [14]:
#####4. Getting N most similiar users for target user

#Generating a pivot table with movie_id as index and user_id as column
user_preference = user_ratings.pivot_table(index=['movie_id'], columns=['user_id'], values = 'rating')

def N_similiar_users (user_id, user_preference):
    # Computing pearson correlation
    corrMatrix_user_preference = user_preference.corr()
    corrMatrix_user_preference = user_preference.corr(method = 'pearson', min_periods = 10)

    #Definig user_id and target index (+1 as python index starts from 0)

    target_location = user_id + 1
    target_user_preference = corrMatrix_user_preference.loc[corrMatrix_user_preference.index.get_loc(target_location)].dropna()

    #Sorting correlation in descending order and defining n users

    target_user_preference.sort_values(inplace = True, ascending = False)
    similiar_users = target_user_preference[1:n_user1]

    print ('\n')
    print(n_user, 'similiar users to userid',user_id, 'alongwith pearson similarity are:')
    print ('\n')
    print(round(similiar_users,2))
    
    return similiar_users


In [15]:
####5. Recommending N recommendations for target user

#merging user_rating and movie data sets
df2 = pd.merge(user_ratings,movies)

#droping unnecessary columns
df2 = df2.drop(['timestamp', 'release_date', 'Blank', 'imdb_url','unknown', 'Action', 'Adventure', 'Animation', 'Children', 'Comedy', 'Crime', 'Documentary', 'Drama', 'Fantasy','Film-Noir', 'Horror', 'Musical', 'Mystery', 'Romance', 'Sci-Fi', 'Thriller', 'War', 'Western'], axis = 1)

#creating pivot table
userRatings = df2.pivot_table(index=['user_id'], columns=['title'], values = 'rating')

def movie_recommendation(user_id, userRatings):
    
    #Generating pearson similarity for movies which had minimum 100 ratings
    corrMatrix = userRatings.corr()
    corrMatrix = userRatings.corr(method = 'pearson', min_periods = 100)

    #Getting index of target userid
    target_user = userRatings.loc[userRatings.index.get_loc(user_id)].dropna()

    #Calculating pearson similarity for all movies with respect to userid
    MovieRecom = pd.Series()
    for i in range(0, len(target_user.index)):
        sims = corrMatrix[target_user.index[i]].dropna()
        sims = round(sims.map(lambda x:x * target_user[i]),3)
        MovieRecom = MovieRecom.append(sims)

    #sorting similarity score
    MovieRecom.sort_values(inplace = True, ascending = False)

    #As there may be multiple rating, grouping movies and adding their similarity score 
    MovieRecom = MovieRecom.groupby(MovieRecom.index).sum()

    #Sorting score in descending order
    MovieRecom.sort_values(inplace = True, ascending = False)

    #printing recommendation
    print ('\n')
    print('Recommended movies for user id', user_id, 'alongwith pearson similarity score are:')
    print('\n')
    print(MovieRecom.head(nrecom))
    
    return MovieRecom



In [23]:
###6. User Interface

print ('Greetings User!!')
print('What do you want to calculate today?')

try:
    
    while True:

        User_choice = int(input('Enter 1 for movie similarity or 2 for user similarity or 3 for movie recommendation for user-     '))

        if User_choice == 1:
            User_choice_movie = int(input('Enter 1 to calculate movie similarity between two movies or 2 to calculate similiar movies for target movie-     '))

            if User_choice_movie == 1:
                from numpy import ndarray
                print ('\n')
                movie_id1 = int(input('Please enter first movie id -    '))
                movie_id2 = int(input('Please enter second movie id -   '))
                print ('\n')
                similarity_btwn_2movies (movie_id1, movie_id2, sim_btwn_movies)

            elif User_choice_movie == 2:
                    print ('\n')
                    movie_id = int(input('Please enter movie id -      '))
                    n_movies = int(input('Please enter number of similiar movies -    '))
                    title = title_from_movie_id(movie_id)
                    similiar_movies_to_target_movie = N_similiar_movies(title)
                    print('\nSimiliar movies for', title, 'are:')
                    print ('\n')
                    print(similiar_movies_to_target_movie)

            else:
                print('Invalid Entry. Please try again!!!')

        elif User_choice == 2:
            User_choice_user = int(input('Enter 1 to calculate user similarity between two users or 2 to calculate similiar users for target user    '))

            if User_choice_user == 1:
                print ('\n')
                userid1 = int(input('Please enter first user id -    '))
                userid2 = int(input('Please enter second user id -   '))
                print(user_similarity_metric(userid1, userid2, df2))



            elif User_choice_user == 2:
                print ('\n')
                user_id = int(input('Please enter user id -      '))
                n_user = int(input('Please enter number of similiar users -     '))
                n_user1 = n_user + 1
                similiar_users_to_target_users = N_similiar_users (user_id, user_preference)


            else:
                print('Invalid Entry. Please try again!!!')

        elif User_choice == 3:
            print ('\n')
            user_id = int(input('Please enter user id -    '))
            nrecom = int(input('Please enter number of recommedations -   '))
            movie_recommendation(user_id, userRatings)


        else:
            print('Invalid Entry. Please try again!!!!!!')

        print('\n')
        Next = input('Do you want to do further calculation? If yes, enter yes, else enter quit.    ')

        if Next == 'quit':
            print('Thank you for your visit. Goodbye!!!')
            break

        elif Next == 'yes':
            print('Great to see you back!!')

        else:
            print('Invalid Entry. Please try again!!!')
except:
    print('Invalid Entry. Please try again!!!')


Greetings User!!
What do you want to calculate today?
Enter 1 for movie similarity or 2 for user similarity or 3 for movie recommendation for user-     3


Please enter user id -    567
Please enter number of recommedations -   7


Recommended movies for user id 567 alongwith pearson similarity score are:


Raiders of the Lost Ark (1981)               71.531
Back to the Future (1985)                    70.163
Fugitive, The (1993)                         63.505
Indiana Jones and the Last Crusade (1989)    62.955
Braveheart (1995)                            61.075
Mission: Impossible (1996)                   59.134
Toy Story (1995)                             56.979
dtype: float64


Do you want to do further calculation? If yes, enter yes, else enter quit.    yes
Great to see you back!!
Enter 1 for movie similarity or 2 for user similarity or 3 for movie recommendation for user-     2
Enter 1 to calculate user similarity between two users or 2 to calculate similiar users for target user 